### **Búsqueda Semántica y RAG local (FastAPI + LangGraph + GraphRAG + LATS)**

Todo el flujo se basa en modelos y servicios **locales** y en frameworks modernos:

- Embeddings locales con `sentence-transformers`.
- Vector store local con FAISS **y opción de Qdrant** como base de datos vectorial.
- LLM local tipo **Llama** servido vía **Ollama**.
- RAG básico con LangChain.
- RAG como grafo con LangGraph.
- Esqueleto de GraphRAG con **Neo4j**.
- API HTTP con FastAPI + métricas para **Prometheus/Grafana**.
- Esqueletos de LATS (Burr + LangGraph).

 Muchos bloques están en modo plantilla (`TODO` o "esqueleto") para que se completen y adapten a su propia infraestructura.


#### **0. Fundamentos: ¿por qué RAG y no solo *fine-tuning*?**

En un escenario real partimos casi siempre de un LLM ya pre-entrenado. Para adaptarlo a un dominio concreto hay dos grandes estrategias:

- **Fine-tuning clásico**: entrenar de nuevo el modelo (total o parcialmente) con ejemplos del nuevo dominio.
- **RAG (Retrieval-Augmented Generation)**: dejar fijo el modelo y conectarlo a una base de conocimiento externa que se consulta en tiempo de inferencia.

**Fine-tuning** es útil cuando:

- queremos cambiar el "comportamiento" del modelo (estilo de respuesta, formato, instrucciones complejas).
- disponemos de miles/millones de ejemplos limpios y etiquetados.
- aceptamos pagar coste de cómputo y de mantenimiento del modelo afinado.

Sus limitaciones:

- actualizar conocimiento requiere volver a entrenar.
- es difícil borrar o aislar información concreta (riesgos legales/cumplimiento).
- puede degradar capacidades generales si el dataset es pequeño o sesgado.

**RAG**, en cambio:

- mantiene fijo el LLM base;
- recupera documentos relevantes (*retrieve*) desde una base de conocimiento;
- y genera la respuesta condicionada por ese contexto (*generate*).

Ventajas de RAG:

- actualizar el sistema = actualizar los documentos (sin re-entrenar el modelo);
- se pueden devolver **citas** y trazabilidad de la fuente;
- más barato de iterar para muchos proyectos "data-centric".

Desventajas de RAG:

- la calidad de la respuesta depende fuertemente del **retriever** y del **chunking**;
- el contexto está limitado por la ventana del modelo;
- hay que diseñar bien la base de conocimiento y los metadatos.

En este cuaderno nos vamos a centrar en **RAG local** (FAISS/Qdrant + LLM vía Ollama) y en sus variantes (GraphRAG, agentes tipo LATS). El *fine-tuning* aparecerá como **complemento** posible, pero no como la herramienta principal.


#### **Ejercicio 1 - RAG vs *Fine-Tuning***

 Considera estos cuatro escenarios:
 
 1. Un *chatbot* para soporte interno de una universidad, que debe responder preguntas sobre reglamentos y documentos que cambian cada semestre.
 2. Un asistente de programación que debe aprender el estilo de código de una empresa concreta.
 3. Un buscador de jurisprudencia legal donde cada respuesta debe venir acompañada de citas exactas a artículos y sentencias.
 4. Un modelo que debe aprender a escribir poesía al estilo de un poeta específico.
 
 **Tareas:**
 - (a) Para cada escenario, indica si usarías **RAG**, **fine-tuning**, o una **combinación**. Justifica en 3-5 líneas.
 - (b) Elige uno de los escenarios donde hayas propuesto RAG y explica:
   - Qué tipo de documentos indexarías.
   - Cada cuánto actualizarías la base de conocimiento.
   - Qué riesgos ves si sólo hicieras *fine-tuning* y no RAG.
 - (c) Redacta en una frase tu propia definición de "RAG" y otra de "fine-tuning" que un compañero de pregrado pueda entender sin leer el cuaderno.


In [ ]:
## Tus respuestas

#### **1. Requisitos de entorno**

Instala estos paquetes en tu entorno local (o WSL/Colab)

In [ ]:
!pip install \
  sentence-transformers \
  faiss-cpu \
  neo4j \
  rank-bm25 \
  langchain \
  langchain-community \
  langchain-qdrant \
  qdrant-client \
  langgraph \
  "fastapi[standard]" uvicorn[standard] \
  prometheus-fastapi-instrumentator prometheus-client \
  "burr[start]"


#### **2. Corpus de ejemplo**

Pequeño corpus basado en *Interstellar*. Sustituye por tu propio conjunto de textos si lo deseas.


**2.1 Pipeline de ingesta y *chunking* en un caso real**

El "corpus de ejemplo" que usamos aquí (`texts = [...]`) simula el resultado de un pipeline más completo que, en producción, suele tener estos pasos:

1. **Ingesta**: leer PDFs, páginas web, Markdown, etc.
2. **Limpieza y normalización**: quitar ruido, headers, pie de página, código HTML innecesario, etc.
3. **Segmentación en *chunks***:
   - dividir los documentos en trozos de longitud razonable (por ejemplo 500-1000 caracteres);
   - añadir un solapamiento (por ejemplo 50-100 caracteres) para no cortar ideas a la mitad.
4. **Enriquecimiento con metadatos**: fuente, autor, fecha, idioma, tipo de documento, nivel de confidencialidad, etc.
5. **Cálculo de embeddings + almacenamiento en una base vectorial**.

En pseudo-código con LangChain podría verse así:

```python
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Ingesta: cargar PDFs desde una carpeta
raw_docs = []
for pdf_path in Path("data/").glob("*.pdf"):
    loader = PyPDFLoader(str(pdf_path))
    raw_docs.extend(loader.load())

# 2-3. Chunking: dividir en trozos con solapamiento
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
)
chunked_docs = splitter.split_documents(raw_docs)

# 4. Resultado listo para embeddings: cada chunk tiene page_content + metadata
print(len(chunked_docs), chunked_docs[0].page_content[:200])


In [ ]:
# Corpus de ejemplo: lista de párrafos cortos
texts = [
    "Interstellar is a science fiction film directed by Christopher Nolan. "
    "It explores themes of space travel, black holes, and the survival of humanity.",
    "The movie relies heavily on theoretical physics, including concepts such as wormholes, "
    "time dilation, and higher-dimensional spaces.",
    "The main character, Cooper, joins a mission through a wormhole near Saturn in search of "
    "habitable planets for humans to colonize.",
    "Interstellar's depiction of a rotating black hole, Gargantua, was praised for its scientific "
    "accuracy thanks to physicist Kip Thorne.",
    "Beyond the science, the film emphasizes emotional connections, especially between Cooper "
    "and his daughter Murph.",
]

len(texts), texts[0][:120]


En este cuaderno usamos un corpus pequeño para centrarnos en las ideas, pero mentalmente puedes imaginar que la lista `texts` proviene de un pipeline como el anterior.

 #### **Ejercicio 2 - Comparando estrategias de *chunking***
 
 Supón que tienes un documento de ~20 páginas en PDF. Vas a probar dos configuraciones de *chunking*:
 
 - Configuración A: `chunk_size = 500`, `chunk_overlap = 50`
 - Configuración B: `chunk_size = 1200`, `chunk_overlap = 200`
 
 **Tareas (modificando el código de la sección 2.1):**
 1. Implementa ambas configuraciones con `RecursiveCharacterTextSplitter`.
 2. Para cada configuración, imprime:
    - Número total de *chunks*.
    - Longitud media de `page_content` (en caracteres).
 3. Comenta:
    - ¿Cuál configuración crees que es mejor para **preguntas muy específicas**?
    - ¿Cuál configuración favorece más preguntas **muy generales**?
    - ¿En cuál configuración esperas ver más "cortes raros" de frases?
 4. Elige una de las dos configuraciones como la que usarías "por defecto" en tu proyecto y justifica en 5-7 líneas.


In [ ]:
## Tus respuestas

**2.2 Conceptos clave: embeddings, vector DB y búsqueda aproximada**

Antes de meternos en FAISS, conviene fijar el vocabulario:

- Un **embedding** es un vector de dimensión `d` (por ejemplo `d = 384` o `768`) que representa el significado de un texto en un **espacio vectorial**.
- Textos "parecidos" se mapean a vectores cercanos; textos "distintos" quedan lejos.

Tipos habituales:

- **Token embeddings**: un vector por token (palabra/sub-palabra).
- **Sentence / document embeddings**: un vector por oración, párrafo o documento (lo que usamos en este cuaderno).

Para comparar vectores podemos usar varias **métricas de distancia/similitud**:

- **Producto punto** (lo que usamos con `IndexFlatIP` de FAISS).
- **Similaridad de coseno**: producto punto entre vectores normalizados.
- **L2 /Distancia Euclidiana**: distancia geométrica clásica.

La búsqueda "ingenua" sería comparar la consulta con **todos** los vectores (búsqueda exacta), pero cuando tenemos millones de documentos esto se vuelve caro. Ahí entran los **índices ANN (Approximate Nearest Neighbors)**:

- estructuras de datos especializadas (IVF, HNSW, etc.) que permiten encontrar vecinos "suficientemente buenos" mucho más rápido;
- muchas bases vectoriales (FAISS, Qdrant, Milvus, etc.) implementan estas técnicas por debajo.

En las siguientes secciones verás primero un índice FAISS simple en memoria, y luego cómo envolver estos embeddings en un `vectorstore` (FAISS/Qdrant) listo para usarse desde un RAG.


#### **Ejercicio 3 -Explorando métricas de distancia en la búsqueda vectorial**
 
 En esta sección usamos un índice FAISS con producto punto/*inner product*.
 
 **Tareas:**
 1. Modifica el código para probar dos configuraciones:
    - (a) Normalizar los embeddings (norma 1) y seguir usando *inner product*.
    - (b) Usar un índice FAISS basado en L2 (`IndexFlatL2`) sin normalización.
 2. Define al menos **3 consultas** diferentes sobre el corpus (ejemplo,  "black hole", "Cooper", "Earth" si usas Interstellar).
 3. Para cada consulta:
    - Muestra los **top-5 documentos** recuperados con (a) y con (b).
    - Señala en cuáles casos te parece que el ranking es mejor con (a) y en cuáles con (b).
 4. Escribe un breve comentario (8-10 líneas) sobre:
    - Cómo afecta la métrica de distancia al resultado.
    - Por qué podrías preferir una u otra en un sistema real.


In [ ]:
## Tus respuestas

#### **3. Embeddings locales y búsqueda semántica con FAISS**

Usamos `sentence-transformers` (por ejemplo `BAAI/bge-small-en-v1.5`) y FAISS para k-NN.

Aquí nos centraremos primero en la parte de **búsqueda densa**: cómo pasar de textos a vectores y cómo hacer *k*-NN sobre ellos. En la siguiente sección verás cómo complementar esto con un modelo **sparse** (BM25) y luego combinarlos en un *retriever* híbrido más robusto.



In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embedding_model_name = "BAAI/bge-small-en-v1.5"
embedder = SentenceTransformer(embedding_model_name)

def get_doc_embeddings_local(texts):
    return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

def get_query_embedding_local(query: str):
    return embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]

doc_embeddings = get_doc_embeddings_local(texts)
dim = doc_embeddings.shape[1]

index = faiss.IndexFlatIP(dim)
index.add(doc_embeddings.astype("float32"))

print("Índice FAISS construido:", doc_embeddings.shape)


#### **3.1 Función de búsqueda semántica**


In [ ]:
def search_semantic(query: str, k: int = 3):
    q_vec = get_query_embedding_local(query).astype("float32")[None, :]
    scores, indices = index.search(q_vec, k)
    scores, indices = scores[0], indices[0]
    results = []
    for s, i in zip(scores, indices):
        results.append({
            "score": float(s),
            "index": int(i),
            "text": texts[i],
        })
    return results

for r in search_semantic("How accurate was the black hole science?", k=3):
    print(f"[score={r['score']:.4f}] idx={r['index']} - {r['text'][:100]}...")


#### **4. Sparse retrieval con BM25 y combinación híbrida**

La búsqueda **sparse** clásica (tipo BM25) no usa embeddings: se basa en las palabras que aparecen en cada documento y en la frecuencia con la que lo hacen.  

En esta sección verás:

- cómo construir un índice BM25 sobre el mismo corpus,
- cómo obtener resultados sólo con BM25,
- y cómo combinar sus *scores* con los de la búsqueda densa anterior para obtener un *retriever* híbrido más robusto.


In [ ]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [t.lower().split() for t in texts]
bm25 = BM25Okapi(tokenized_corpus)

def search_bm25(query: str, k: int = 3):
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    ranked_idx = np.argsort(scores)[::-1][:k]
    results = []
    for i in ranked_idx:
        results.append({
            "score": float(scores[i]),
            "index": int(i),
            "text": texts[i],
        })
    return results

for r in search_bm25("black hole and physics", k=3):
    print(f"[bm25={r['score']:.4f}] idx={r['index']} - {r['text'][:100]}...")


In [ ]:
def _normalize(arr):
    arr = np.array(arr, dtype="float32")
    if arr.max() == arr.min():
        return np.ones_like(arr)
    return (arr - arr.min()) / (arr.max() - arr.min())

def search_hybrid(query: str, k: int = 3, alpha: float = 0.5):
    # Puntajes densos
    q_vec = get_query_embedding_local(query).astype("float32")[None, :]
    dense_scores, _ = index.search(q_vec, len(texts))
    dense_scores = dense_scores[0]
    #Puntaje sparse
    tokenized_query = query.lower().split()
    sparse_scores = np.array(bm25.get_scores(tokenized_query), dtype="float32")

    d_norm = _normalize(dense_scores)
    s_norm = _normalize(sparse_scores)
    fused = alpha * d_norm + (1 - alpha) * s_norm
    ranked_idx = np.argsort(fused)[::-1][:k]

    results = []
    for i in ranked_idx:
        results.append({
            "score": float(fused[i]),
            "index": int(i),
            "text": texts[i],
        })
    return results

for r in search_hybrid("wormholes and relativity", k=3, alpha=0.6):
    print(f"[hybrid={r['score']:.4f}] idx={r['index']} - {r['text'][:100]}...")


 #### **Ejercicio 4-Comparando *retrievers*: BM25 vs denso vs híbrido**
 
 En esta sección tienes implementados:
 
 - Un *retriever* **denso** (embeddings + FAISS).
 - Un *retriever* **sparse** (BM25).
 - Una función de búsqueda **híbrida** que combina ambas señales.
 
 **Tareas:**
 1. Elige al menos **5 consultas** sobre tu corpus.
 2. Para cada consulta, ejecuta:
    - Sólo BM25.
    - Sólo denso.
    - Híbrido.
 3. Marca manualmente, para cada consulta, qué documentos son "claramente relevantes" (pueden ser 1-3 por consulta).
 4. Calcula a mano para cada *retriever*:
    - **Recall@3** (¿aparece algún doc relevante en el top-3?).
    - Opcional: la posición del primer documento relevante.
 5. Comenta:
    - ¿En qué tipo de consultas gana BM25? (por ejemplo, con palabras muy poco frecuentes, nombres, etc.)
    - ¿En qué tipo de consultas gana el denso?
    - ¿El híbrido aporta algo o se parece demasiado a uno de los dos?


In [ ]:
## Tus respuestas

#### **5. RAG básico con LangChain (Ollama + FAISS/Qdrant)**

Construimos un pipeline sencillo: `Retriever + LLM`.

En este cuaderno asumiremos un **stack local realista**:

- **LLM** local vía **Ollama** (por ejemplo un modelo `llama3` o similar).
- **Vector store** por defecto en memoria con **FAISS**, y opcionalmente **Qdrant** como base de datos vectorial persistente.
- Más adelante conectaremos esto con:
  - **LangGraph** para orquestar el flujo RAG como grafo.
  - **Neo4j** (esqueleto) para GraphRAG.
  - **FastAPI + Prometheus/Grafana** para exponer el RAG como servicio observable.


#### **5.1 Estrategias de *prompting* en RAG**

Una vez que tenemos un `Retriever`, el siguiente reto es **cómo** pasar el contexto recuperado al LLM. Hay varios patrones típicos:

1. **Stuffing (todo en un solo *prompt*)**

   - Se concatenan los `k` chunks recuperados en un único bloque de contexto.
   - Ventajas: simple, fácil de depurar.
   - Limitaciones: si el contexto es grande puedes chocar con la ventana del modelo y "ahogar" la pregunta.

2. **Map-Reduce**

   - Paso *map*: el LLM genera un pequeño resumen/respuesta parcial por cada chunk relevante.
   - Paso *reduce*: otro *prompt* combina esos resúmenes en una respuesta final.
   - Útil cuando recuperas muchos documentos o cuando cada doc es largo.

3. **Refine**

   - Se procesa el primer chunk y se obtiene una respuesta inicial.
   - Para cada chunk siguiente se llama al LLM con "respuesta actual + nuevo contexto" y se pide refinar/mejorar la respuesta.
   - Es una especie de acumulación paso a paso.

4. **Citations / respuestas con fuentes**

   - El *prompt* pide explícitamente citar las fuentes (por ejemplo `[doc_3]`, `[página 5]`) y nunca inventarlas.
   - Después puedes mapear esas etiquetas a URLs, títulos de archivo, etc., para mostrar trazabilidad al usuario.

En este cuaderno, la función `simple_rag` implementa un patrón de tipo **stuffing**. Más adelante puedes extenderla para añadir variantes *map-reduce* o *refine* usando cadenas compuestas de LangChain o grafos más elaborados en LangGraph.


In [ ]:
from langchain_community.vectorstores import FAISS as LC_FAISS
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings

# Soporte opcional para Qdrant como vector DB
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

# LLM local vía Ollama (por ejemplo llama3)
from langchain_community.llms import Ollama
import os

# Embeddings para el RAG (puedes reutilizar el mismo modelo que arriba)
emb_model_name = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=emb_model_name)

docs = [Document(page_content=t, metadata={"id": i}) for i, t in enumerate(texts)]

# Flag para elegir FAISS (in-memory) o Qdrant (persistente)
USE_QDRANT = False  # ponlo en True si tienes Qdrant corriendo en localhost:6333

if USE_QDRANT:
    # Asume Qdrant en Docker: `docker run -p 6333:6333 qdrant/qdrant`
    qdrant_client = QdrantClient(
        url=os.getenv("QDRANT_URL", "http://localhost:6333"),
        api_key=os.getenv("QDRANT_API_KEY", None),
    )
    collection_name = os.getenv("QDRANT_COLLECTION", "chapter8_semantic_search")

    vectorstore = QdrantVectorStore.from_documents(
        docs,
        embeddings,
        client=qdrant_client,
        collection_name=collection_name,
    )
else:
    vectorstore = LC_FAISS.from_documents(docs, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LLM local (Ollama)
# Requisitos:
# 1) Instalar Ollama (https://ollama.ai) y arrancar el servidor local.
# 2) Ejecutar: `ollama pull llama3` (u otro modelo que prefieras).
# 3) Ajustar el nombre del modelo aquí:
llm = Ollama(
    model=os.getenv("OLLAMA_MODEL", "llama3"),
    temperature=0.1,
)

def simple_rag(question: str) -> str:
    """
    RAG mínimo: recupera k documentos relevantes y le pasa el contexto al LLM local.
    """
    rel_docs = retriever.invoke(question)
    context = "\n\n".join(d.page_content for d in rel_docs)
    prompt = (
        "You are an assistant that answers questions based ONLY on the following context.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        "If the answer is not in the context, say 'I don't know'."
    )
    raw_answer = llm.invoke(prompt)
    # Ollama (vía LangChain) ya devuelve un string
    return str(raw_answer)

# Ejemplo (cuando tengas Ollama corriendo):
# print(simple_rag("How accurate was the science of the black hole?"))


 #### **Ejercicio 5-Probando diferentes estrategias de *prompting* en RAG**
 
 Usando el `Retriever` de esta sección, diseña tres variantes de cadena RAG:
 
 1. **Stuffing**:
    - Concatena los `k` chunks en un solo contexto y llama al LLM una sola vez.
 2. **Map-Reduce**:
    - Paso *map*: para cada chunk relevante, pide al LLM un mini-resumen o mini-respuesta.
    - Paso *reduce*: combina esos resúmenes en una respuesta final.
 3. **Con citas**:
    - Pide al LLM que responda pero citando explícitamente `[doc_i]` o `[chunk_j]` sin inventarse fuentes.
 
 **Tareas:**
 1. Elige **3 preguntas** sobre tu corpus.
 2. Para cada estrategia (stuffing, map-reduce, citas):
    - Guarda la respuesta del modelo.
    - Registra cuántos **tokens** approx. consumiste (puedes estimar por longitud de *prompt* + respuesta).
 3. Compara:
    - ¿Cuál estrategia dio respuestas más completas?
    - ¿Cuál consume más contexto?
    - ¿Cuál te gusta más para un sistema que deba ser explicable ante un "auditor"?
 4. Escribe 5-8 líneas de reflexión sobre el trade-off entre **calidad**, **coste** y **explicabilidad**.


In [ ]:
## Tus respuestas

#### **5.2 Diseño de colecciones y *drift* de embeddings**

Cuando pasamos de un tutorial a un sistema real, ya no tenemos "un solo vectorstore", sino varias **colecciones/namespaces** con políticas distintas. Algunas decisiones típicas:

- Separar colecciones por:
  - **cliente** o proyecto,
  - **idioma**,
  - **tipo de documento** (manuales, tickets, políticas internas),
  - **nivel de confidencialidad**.
- Usar metadatos (`metadata={...}`) para poder filtrar en el `retriever`:
  - por ejemplo, buscar sólo en `{"idioma": "es", "tipo": "manual"}`.

Con Qdrant (y otras bases vectoriales) es buena práctica **versionar** las colecciones, por ejemplo:

- `docs_v1_bge_small`
- `docs_v2_bge_m3`

Esto ayuda a manejar el llamado **embedding drift**:

- cambias de modelo de embeddings (por calidad, coste, idioma);
- o cambia el dominio de tus datos con el tiempo.

Estrategias básicas para el *drift*:

- mantener colecciones antiguas **sólo lectura** mientras reindexas en una colección nueva;
- registrar en los metadatos qué versión de modelo generó cada embedding;
- definir una política clara de "deprecación" de colecciones viejas.

En el bloque de código anterior puedes imaginar que el `collection_name` de Qdrant incluye ya la versión del modelo de embeddings que estás usando.


 #### **Ejercicio 6-Diseñando colecciones y manejando *drift* de embeddings**
 
 Imagina que trabajas en una empresa que quiere un RAG para:
 - Documentación interna de DevSecOps (español/inglés).
 - Políticas de RR.HH. (sólo español).
 - Manuales de producto para clientes externos (multilingüe).
 
 Además, planean cambiar de modelo de embeddings en 6 meses.
 
 **Tareas:**
 1. Propón un esquema de **colecciones/namespaces** (por ejemplo, nombres de colecciones en Qdrant) y describe en 5-6 líneas las reglas para:
    - qué va en cada colección;
    - quién puede consultar cada colección (alto nivel: interno vs externo).
 2. Diseña los **metadatos mínimos** que almacenarías por chunk (`metadata={...}`).
 3. Describe un plan para manejar el **cambio de modelo de embeddings** ("v1" - "v2"):
    - ¿Cómo versionarías las colecciones?
    - ¿Cómo migrarías a la nueva versión minimizando cortes?
 4. Opcional: Diseña un pequeño "playbook" en 4-5 pasos de qué hacer si detectas que la nueva versión empeora mucho el rendimiento.


In [ ]:
## Tus respuestas

#### **6. RAG como grafo con LangGraph**

El mismo RAG anterior se puede expresar como un grafo de estados con LangGraph.

Pasar a un grafo nos permite:

- hacer explícito el flujo de estados (entrada - recuperación - generación),
- añadir fácilmente pasos intermedios (post-procesado, re-ranqueo, verificación, etc.),
- y conectar luego este grafo con APIs (FastAPI) o con agentes más complejos (LATS, GraphRAG).

En el bloque de código siguiente definiremos un estado mínimo (`RAGState`) y dos nodos (`retrieve` y `generate`) para capturar el RAG básico como grafo.



In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

class RAGState(TypedDict, total=False):
    question: str
    retrieved_docs: List[Document]
    answer: str

def retrieve_node(state: RAGState) -> RAGState:
    question = state["question"]
    docs_ = retriever.invoke(question)
    return {**state, "retrieved_docs": docs_}

def generate_node(state: RAGState) -> RAGState:
    if llm is None:
        raise ValueError("Define 'llm' antes de ejecutar el grafo RAG.")
    docs_ = state.get("retrieved_docs", [])
    context = "\n\n".join(d.page_content for d in docs_)
    question = state["question"]
    prompt = (
        "You are an assistant that answers questions based ONLY on the given context.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}"
    )
    answer = llm.invoke(prompt)
    return {**state, "answer": answer}

g_builder = StateGraph(RAGState)
g_builder.add_node("retrieve", retrieve_node)
g_builder.add_node("generate", generate_node)
g_builder.set_entry_point("retrieve")
g_builder.add_edge("retrieve", "generate")
g_builder.add_edge("generate", END)

rag_graph = g_builder.compile()

# Ejemplo (cuando tengas LLM):
# result = rag_graph.invoke({"question": "What are the main scientific themes in Interstellar?"})
# print(result["answer"])


#### **7. FastAPI + LangGraph: servicio RAG (con métricas Prometheus)**

API mínima `POST /rag/query` que invoca el grafo `rag_graph`.

Además, instrumentamos FastAPI con **prometheus-fastapi-instrumentator** para exponer un endpoint
`/metrics` que pueda ser scrapeado por Prometheus y visualizado en Grafana.


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

# Observabilidad con Prometheus
from prometheus_fastapi_instrumentator import Instrumentator
from prometheus_client import Counter

app = FastAPI(title="Local RAG with LangGraph", version="0.1.0")

# Instrumentar FastAPI: añade /metrics automáticamente
Instrumentator().instrument(app).expose(app)

# Métrica personalizada: número de consultas RAG
RAG_REQUESTS = Counter(
    "rag_requests_total",
    "Total de consultas al endpoint /rag/query",
    ["status"],
)

class RAGQuery(BaseModel):
    question: str

class RAGResponse(BaseModel):
    answer: str
    documents: List[str]

@app.post("/rag/query", response_model=RAGResponse)
def rag_endpoint(payload: RAGQuery):
    if llm is None:
        RAG_REQUESTS.labels(status="error_no_llm").inc()
        raise ValueError("Define 'llm' antes de levantar la API RAG.")

    question = payload.question
    result = rag_graph.invoke({"question": question})
    answer = result["answer"]
    docs_ = result.get("retrieved_docs", [])
    docs_text = [d.page_content for d in docs_]

    RAG_REQUESTS.labels(status="ok").inc()
    return RAGResponse(answer=str(answer), documents=docs_text)

"""
Ejemplo para lanzar el servidor en consola:

uvicorn nombre_de_este_archivo:app --reload --port 8000

curl -X POST "http://localhost:8000/rag/query" \
  -H "Content-Type: application/json" \
  -d '{"question": "How accurate was the black hole science?"}'

# Prometheus:
#   - Configura un job que apunte a localhost:8000/metrics
# Grafana:
#   - Añade Prometheus como datasource y crea dashboards con las métricas
#     http_request_duration_seconds y rag_requests_total.
"""


#### **7.1 Evaluación de la calidad de un sistema RAG**

Además de métricas de uso (latencia, errores, `rag_requests_total` en Prometheus), necesitamos métricas para evaluar **qué tan bien responde** nuestro sistema RAG. Hay dos niveles:


##### a) Evaluación del *retriever*

Partimos de un conjunto de prueba con pares *(pregunta, documentos relevantes)*. Algunas métricas:

- **Recall@k**  
  Proporción de preguntas en las que, entre los `k` documentos recuperados, aparece al menos un documento marcado como relevante.

- **MRR (Mean Reciprocal Rank)**  
  Mide en promedio en qué posición aparece el primer documento relevante. Penaliza que el documento correcto esté "enterrado" al final de la lista.

- **nDCG (normalized Discounted Cumulative Gain)**  
  Extiende la idea anterior permitiendo varios niveles de relevancia (muy relevante, algo relevante, etc.) y dando más peso a los primeros puestos del ranking.

Estas métricas son independientes del LLM: sólo miran la calidad de la recuperación.


##### b) Evaluación de las respuestas generadas

Suponiendo que tenemos respuestas de referencia:

- **EM (Exact Match)**: porcentaje de respuestas que coinciden exactamente con la referencia (útil en QA muy estructurado).
- **F1 de *token overlap***: mide solapamiento de palabras entre la respuesta del modelo y la referencia (más tolerante a variaciones de estilo).

En tareas abiertas es habitual usar un **LLM-as-a-judge**:

1. Se pasa al "juez" la pregunta, la respuesta del sistema y, opcionalmente, una respuesta de referencia.
2. Se le pide puntuar según un criterio (exactitud, cobertura, citación correcta, etc.).
3. Se promedian las puntuaciones sobre el conjunto de prueba.

En código, suele verse como una función `score_answer(question, answer, reference) - score de 1 a 5`.  
Este tipo de evaluación debe diseñarse con cuidado (instrucciones claras al juez, muestreo manual para detectar sesgos), pero es práctico cuando las respuestas son largas y semánticamente ricas.


 #### **Ejercicio 7-Implementando métricas de evaluación**
 
 Vas a construir un mini *benchmark* de RAG para tu propio corpus.
 
 **Tareas:**
 1. Define un conjunto de al menos **5 pares**:
    - pregunta `q_i`;
    - conjunto de documentos relevantes `D_i` (índices de tus textos/chunks);
    - una respuesta de referencia corta `a_i` (2-4 líneas).
 2. Implementa en código funciones para:
    - `recall_at_k(queries, retriever, gold_docs, k=3)`;
    - `mrr_at_k(queries, retriever, gold_docs, k=10)`.
 3. Ejecuta las métricas para:
    - el *retriever* denso;
    - el *retriever* híbrido.
 4. Implementa funciones simples para:
    - **Exact Match (EM)** (ignora mayúsculas y puntuación básica);
    - **F1 de solapamiento de tokens** entre respuesta del modelo y `a_i`.
 5. Comenta:
    - ¿Hay correlación entre "buen *retrieval*" y "buena respuesta generada" en tu experimento?
    - ¿En qué casos el LLM respondió bien aunque el *retriever* falló?
    - ¿En qué casos el *retriever* dio buen contexto pero el LLM alucinó?


In [ ]:
## Tus respuestas

 #### **Ejercicio 8 - Observabilidad básica para un servicio RAG**
 
 El servicio FastAPI ya expone:
 - un endpoint `POST /rag/query`;
 - métricas instrumentadas con `prometheus-fastapi-instrumentator` y un `Counter`.
 
 **Tareas:**
 1. Añade una métrica de tipo **Histogram** o **Summary** para:
    - latencia de las peticiones RAG (por ejemplo, `rag_request_latency_seconds`).
 2. Etiqueta la métrica (labels) al menos con:
    - `status` (success/error);
    - opcional: `retrieval_strategy` (dense/sparse/hybrid) si tu código lo soporta.
 3. Usa un pequeño script o el archivo de tráfico de demo para enviar:
    - ≥ 20 peticiones exitosas;
    - algunas peticiones que fuercen error (por ejemplo, tiempo de espera excesivo, query inválida, etc.).
 4. En Prometheus:
    - Muestra la latencia media de los últimos 5 minutos.
    - Filtra por `status` para comparar éxito vs error.
 5. Escribe 5-8 líneas explicando:
    - Qué *SLO* razonable pondrías a la latencia de tu endpoint RAG.
    - Qué tipo de alerta (regla) te gustaría configurar si la latencia se dispara o la tasa de errores sube mucho.


In [ ]:
## Tus respuestas

#### **8. Esqueleto de GraphRAG**

GraphRAG combina vector store + base de datos de grafos. Aquí sólo dejamos el esqueleto conceptual para integrarlo con Neo4j, FalkorDB, etc.

Mientras que el RAG "clásico" consulta sólo la base vectorial, en GraphRAG podemos:

- recuperar nodos relacionados (entidades, conceptos, documentos) a través de aristas,
- enriquecer el contexto con relaciones explícitas,
- y aplicar algoritmos de recorrido de grafos antes de llamar al LLM.

Esto abre la puerta a un razonamiento más estructurado, que conectaremos más adelante con agentes tipo LATS.



In [ ]:
# Esqueleto conceptual (no ejecutable tal cual) para GraphRAG con Neo4j
# La idea: combinar el retriever vectorial (FAISS/Qdrant) con un grafo de conocimiento en Neo4j.

from typing import List
import os
from neo4j import GraphDatabase

NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def _cypher_neighbors(tx, entity_id: str) -> List[str]:
    query = """    MATCH (e:Entity {id: $entity_id})-[:RELATED_TO*1..2]-(n)
    RETURN DISTINCT n.summary AS text
    LIMIT 20
    """
    result = tx.run(query, entity_id=entity_id)
    return [r["text"] for r in result if r["text"]]

def graph_rag(question: str) -> str:
    """
    1) Usar el retriever vectorial para encontrar documentos/contextos ancla.
    2) Extraer de sus metadatos un id de entidad (por ejemplo, `metadata["id"]` o `metadata["entity_id"]`).
    3) Ampliar el contexto con nodos vecinos en Neo4j (1-2 hops).
    4) Enviar contexto + pregunta al LLM local (Ollama).
    """
    if llm is None:
        raise ValueError("Define 'llm' para usar graph_rag.")

    base_docs = retriever.invoke(question)
    base_context = "\n\n".join(d.page_content for d in base_docs)

    # Ejemplo simple: usar los ids de metadatos como entity_id
    entity_ids = [str(d.metadata.get("id")) for d in base_docs if "id" in d.metadata]

    graph_texts: List[str] = []
    if entity_ids:
        with driver.session() as session:
            for eid in entity_ids:
                graph_texts.extend(session.execute_read(_cypher_neighbors, entity_id=eid))

    graph_context = "\n\n".join(graph_texts)

    full_context = base_context
    if graph_context:
        full_context += "\n\n--- Graph context (Neo4j) ---\n\n" + graph_context

    prompt = (
        "You are a GraphRAG assistant. Answer based on the following hybrid context "
        "(vector store + Neo4j graph).\n\n"
        f"Context:\n{full_context}\n\n"
        f"Question: {question}\n"
        "If you are not sure, say that you don't know."
    )
    raw_answer = llm.invoke(prompt)
    return str(raw_answer)

# NOTA: este esqueleto asume que:
# - Ya cargaste entidades y relaciones en Neo4j.
# - Tus documentos tienen metadatos que permiten mapearlos a nodos del grafo.


#### **9. Esqueleto de LATS con Burr**

LATS (Language Agent Tree Search) se puede implementar como máquina de estados en Burr.

Este esqueleto ilustra la idea de tratar un agente como una **máquina de estados** donde cada acción (planificar, recuperar, evaluar, decidir siguiente paso) es un nodo del grafo. En la siguiente sección verás una variante similar implementada directamente con LangGraph.


In [ ]:
from burr.core import action, State, ApplicationBuilder

class QAState(State):
    """Estado del agente de QA/RAG."""
    pass

@action(reads=["question"], writes=["subquestions"])
def plan_step(state: QAState) -> QAState:
    q = state["question"]
    # TODO: llamar a un LLM para descomponer la pregunta en subpreguntas
    subqs = [q]  # placeholder
    return state.update(subquestions=subqs)

@action(reads=["subquestions"], writes=["candidate_answers"])
def retrieve_step(state: QAState) -> QAState:
    subqs = state["subquestions"]
    cands = []
    for sq in subqs:
        # TODO: usar simple_rag / graph_rag / search_semantic + LLM
        # ans = simple_rag(sq)
        ans = f"TODO: answer for '{sq}'"
        cands.append({"question": sq, "answer": ans})
    return state.update(candidate_answers=cands)

@action(reads=["candidate_answers"], writes=["final_answer"])
def evaluate_step(state: QAState) -> QAState:
    cands = state["candidate_answers"]
    # TODO: usar LLM-as-a-judge para elegir la mejor combinación
    if cands:
        final = cands[0]["answer"]
    else:
        final = "No answer found."
    return state.update(final_answer=final)

lats_app = (
    ApplicationBuilder()
    .with_actions(plan_step, retrieve_step, evaluate_step)
    .with_transitions(
        ("plan_step", "retrieve_step"),
        ("retrieve_step", "evaluate_step"),
    )
    .with_state(question=None)
    .with_entrypoint("plan_step")
    .build()
)

# Ejemplo (cuando conectes LLM y RAG reales):
# *_, end_state = lats_app.run(
#     halt_after=["evaluate_step"],
#     inputs={"question": "How accurate is the science in Interstellar?"},
# )
# print("Final answer:", end_state["final_answer"])


#### **10. LATS / tree-search con LangGraph**

Versión simplificada de búsqueda en árbol (tipo LATS) usando LangGraph directamente.


In [ ]:
from typing import TypedDict

class SearchNode(TypedDict, total=False):
    hypothesis: str
    score: float

class LATSState(TypedDict, total=False):
    question: str
    frontier: list[SearchNode]
    best_answer: str

def expand_node(state: LATSState) -> LATSState:
    q = state["question"]
    frontier = state.get("frontier", [])
    # TODO: usar LLM para generar nuevas hipótesis
    new_h = {"hypothesis": f"TODO: hypothesis for '{q}'", "score": 0.0}
    frontier.append(new_h)
    return {**state, "frontier": frontier}

def score_node(state: LATSState) -> LATSState:
    frontier = state.get("frontier", [])
    if not frontier:
        return state
    # TODO: usar LLM o función de reward para puntuar
    frontier[-1]["score"] = 1.0
    best = max(frontier, key=lambda n: n.get("score", 0.0))
    return {**state, "frontier": frontier, "best_answer": best["hypothesis"]}

search_builder = StateGraph(LATSState)
search_builder.add_node("expand", expand_node)
search_builder.add_node("score", score_node)
search_builder.set_entry_point("expand")
search_builder.add_edge("expand", "score")
search_builder.add_edge("score", END)  # en un LATS real se haría un bucle

lats_graph = search_builder.compile()

# result = lats_graph.invoke({"question": "How precise was the black hole depiction?"})
# print("Best hypothesis:", result.get("best_answer"))


 #### **Ejercicio 9-LATS (Language Agent Tree Search) sobre el grafo de LangGraph**
 
 En esta sección has visto un esqueleto de LATS donde el agente:
 - descompone la pregunta en sub-preguntas (*plan*),
 - recupera contexto para cada sub-pregunta (*retrieve*),
 - genera respuestas parciales o candidatas (*answer*),
 - y recorre un árbol de posibilidades.
 
**Parte A-Preguntas conceptuales**
 
 1. **RAG lineal vs LATS en árbol**  
    Explica con tus palabras cómo LATS extiende un RAG "lineal" (retrieve -> generate) a un proceso de **búsqueda en árbol**:
    - ¿Qué papel juegan los pasos de `plan`, `retrieve` y `answer` (o `evaluate`, si lo tienes) en el grafo de LATS del cuaderno?  
    - ¿Por qué puede ser útil tener varios "nodos de respuesta candidata" en el árbol en lugar de una única respuesta directa del LLM?
 
 2. **Heurísticas y verificación**  
    Imagina que quieres robustecer el esqueleto de LATS añadiendo una etapa explícita de **verificador** (*LLM-as-a-judge*) antes de elegir la respuesta final:  
    - ¿Qué tipo de criterio o *score* (por ejemplo, de 1 a 5) le pedirías al verificador que asigne a cada respuesta candidata?  
    - Menciona al menos **dos riesgos** si esa heurística o verificador están mal diseñados (por ejemplo, aceptar respuestas alucinadas, rechazar respuestas correctas, etc.).
 
 
**Parte B- Modificación práctica del grafo LATS**
 
 A partir del grafo de la celda `LATS/tree-search con LangGraph`, extiende el código para añadir un nodo de verificación.
 
 3. **Ampliar el estado de LATS**  
    - Localiza la definición de tu estado (por ejemplo `class LATSState(TypedDict, ...)` o similar).  
    - Añade uno o más campos para almacenar evaluación del verificador, por ejemplo:
      - `scores: dict[str, float]` (score por respuesta candidata), o  
      - `best_answer: str` y `best_score: float`, o  
      - `accepted: bool` si decides usar un simple "aceptar/rechazar".
 
 4. **Implementar un nodo `verify_step`**  
    - Crea una nueva función/nodo, por ejemplo:
      ```python
      @app.node
      def verify_step(state: LATSState) - LATSState:
          \"\"\"Usa el LLM como juez sobre las respuestas candidatas.\"\"\"
          # 1. Construye un prompt del tipo:
          #    - pregunta original
          #    - respuestas candidatas (por ejemplo, una lista numerada)
          #    - instrucción: "evalúa cada respuesta de 1 a 5 según exactitud y uso correcto del contexto"
          # 2. Llama al LLM en modo "juez" y parsea la salida
          # 3. Actualiza el estado con los puntajes (scores) y/o selecciona la mejor respuesta
          return state
      ```
    - No necesitas que el nodo sea perfecto, basta con que:
      - lea del estado las respuestas candidatas,
      - devuelva el estado enriquecido con algún tipo de score o bandera.
 
 5. **Conectar `verify_step` en el grafo**  
    - Modifica la definición de tu grafo LATS para que, después del nodo que genera las respuestas (por ejemplo `answer_step`):
      - se ejecute `verify_step` antes de llegar al estado final.  
    - Diseña una regla simple:
      - si el `best_score` (o score medio) es **mayor o igual a un umbral** `τ` (por ejemplo 4.0), el grafo termina y devuelve esa respuesta;  
      - si el score es **menor que `τ`**  vuelve a un nodo anterior (por ejemplo `plan_step` o `retrieve_step`) para refinar la descomposición o recuperar más contexto.
 
 6. **Comparación rápida LATS vs RAG lineal**  
    - Elige **3 preguntas**:
      1. Una sencilla que no requiera descomposición.
      2. Una que naturalmente se divida en 2-3 sub-preguntas.
      3. Una ambigua o incompleta.
    - Para cada pregunta:
      - ejecuta tu RAG "lineal" y guarda la respuesta,
      - ejecuta tu LATS extendido con `verify_step` y guarda la respuesta;
      - anota cuántas llamadas al LLM se hicieron en cada caso (aproximado).
    - Escribe 6-8 líneas comentando:
      - en cuál de las preguntas LATS aporta una mejora clara frente al RAG lineal;
      - en qué casos el coste (más llamadas al LLM, más complejidad) no se justifica.


In [ ]:
## Tus respuestas